In [31]:
"Source Medium articles, Youtube, Coursera, most important Stackoverflow"

'Source Medium articles, Youtube, Coursera, most important Stackoverflow'

In [1]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import os
import pickle
import pandas as pd
import numpy as np
from collections import Counter
import re
import io
from nltk.corpus import stopwords
from sklearn import svm
from sklearn.model_selection import train_test_split

In [2]:
path_training_data = os.getcwd()+'/training_data.tsv'
path_test_data=os.getcwd()+'/evalution_data.txt'

In [3]:
## Regex model

data = pd.read_csv(path_training_data, header=0,delimiter="\t",quoting=3)

list_sent= list(data['sent'])
list_labels = list(data['label'])

mega_list_label=[]
for idx,label in enumerate(list_labels):
    if label=='Not Found':
        pass
    else:
        mega_list_label.append([list_sent[idx],label])

In [4]:
# pattern with words on both side 

some_list=[]
for i in mega_list_label:
    name = i[1]
    name = re.sub('[^A-Za-z0-9]+', ' ', name)
    sub = '(\w*)\W*('+name+')\W*(\w*)'
    str1 = i[0]
    for i in re.findall(sub, str1, re.I):
        some_list.append ([" ".join([x for x in i if x != ""]),name])

some_pattern_list=[]
for double_list in some_list:
    if double_list[1] in double_list[0]:
        some_pattern_list.append(double_list[0].replace(double_list[1], " "))
    else:
        pass
        
pattern_list1=((Counter(some_pattern_list)).most_common())
pattern_list=[tup[0] for tup in pattern_list1]

In [5]:
# patterns with two words on the left 
some_new_list=[]
for i in mega_list_label:
    name = i[1]
    name = re.sub('[^A-Za-z0-9]+', ' ', name)
    sub = '(\w*)\W*(\w*)\W*('+name+')'
    str1 = i[0]
    for i in re.findall(sub, str1, re.I):
        some_new_list.append ([" ".join([x for x in i if x != ""]),name])  
some_pattern_list_new=[]
for double_list in some_new_list:
    if double_list[1] in double_list[0]:
        some_pattern_list_new.append(double_list[0].replace(double_list[1], " "))
    else:
        pass
pattern_list2=((Counter(some_pattern_list_new)).most_common())
pattern_list_2=[tup[0] for tup in pattern_list2]
len_2_words=int((len(pattern_list_2))*.20)

In [7]:
    ######## Creating the regex_matcher.py file ########

    target = io.open('regex_matcher.py','a', encoding="utf-8")
    target.truncate()
    target.write('import re')
    target.write('\n')
    target.write('def main_regex_matcher(path_test):')
    target.write('\n')
    target.write(
        '''
    master_list=[]
    sent_list=[]
    data = open(path_test,'r')
    for text in data:
    found = 0
    small_master_list=[]
        '''
    )    
    target.write('\n')
    target.close()
    for pat in pattern_list:
        boundaries=pat.split()
        if len(boundaries)==2:
            target = io.open('regex_matcher.py','a', encoding="utf-8")
            target.write('        m = re.search('+"' "+boundaries[0]+' '+'(.+?)'+' '+boundaries[1]+" '"+', text)')
            target.write('\n')
            target.write('        if m:')
            target.write('\n')
            target.write('            found = m.group(1)')
            target.write('\n')
            target.write('            small_master_list.append(found)')
            target.write('\n')
            target.close()
        else:
            pass
    for pat in pattern_list_2[:len_2_words]:
        boundaries=pat.split()
        if len(boundaries)==2:
            target = io.open('regex_matcher.py','a', encoding="utf-8")
            target.write('        m = re.search('+"' "+boundaries[0]+' '+boundaries[1]+' (.*)'+"'"+', text)')
            target.write('\n')
            target.write('        if m:')
            target.write('\n')
            target.write('            found = m.group(1)')
            target.write('\n')
            target.write('            small_master_list.append(found)')
            target.write('\n')
            target.close()
        else:
            pass

    target = open('regex_matcher.py', 'a')
    target.write(
        '''
        if found==0 : # if none of the patterns match give it as Not found
            small_master_list.append('Not Found')
        master_list.append(small_master_list)

    no_no_words=['on','for','to','at','by'] # list of words which if occured in the output will be penalised while giving a score
    final_output=[]
    ############ selecting one from the options
    for options in master_list:
        if len(options)==1:
            final_output.append(options[0])#if only one pattern extracted use it 
        else:
            sent_score_list=[] # else score all the options to select the best one
            for option in options:
                l=option.split()
                score = 0
                score = len(l)
                for word in l:
                    if word in no_no_words:
                        score = score -3 #penalise the no_no_words
                    else:
                        pass
                sent_score_list.append(score)
            m = max(sent_score_list)
            indx=[i for i, j in enumerate(sent_score_list) if j == m] # returns a list of all the index which have max score
            index = indx[-1]#pick the last element as an index 
            final_output.append(options[index])

    tups=zip(sent_list, final_output)
    final_list = [list(l) for l in tups]

    for small_list in final_list:
        target = open("output.txt", "a")
        target.write('\\n')
        target.write(' '.join(small_list[0].split())+'\\t'+small_list[1])
        target.write('\\n')
        target.close()
    return(final_output)
    ''')
    target.write('\n')
    target.close()

In [9]:
import regex_matcher

print('inside_extration....This might take a while')
extractions_from_regex=regex_matcher.main_regex_matcher(path_test_data)
print('extracted')

inside_extration....This might take a while
extracted


In [10]:
#saving the output in the file 
with io.open('output.txt','w+',encoding="utf-8") as f:
    f.truncate()
    for item in extractions_from_regex:
        f.write("%s\n" % item)
    f.close()

In [11]:
## label classifier

data = pd.read_csv(('training_data.tsv'), header=0,delimiter="\t",quoting=3)
data.dropna(inplace=True) #drop NaN value
data.reset_index(drop=True, inplace=True)
data.loc[data['label'] != 'Not Found' , 'label'] = 'Found' #to change data in only found and not found

X = data['sent']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=0)

clean_train_data = []
print("cleaning data\n")
for i in (X_train):
    clean_train_data.append(i)

cleaning data



In [12]:
#Bag of words   
vectorizer = CountVectorizer(ngram_range=(1,2))
train_data_features = vectorizer.fit_transform(clean_train_data)
np.asarray(train_data_features)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(train_data_features)

clf = svm.LinearSVC(loss='hinge').fit(X_train_tfidf,y_train  )

f = open('my_classifier.pickle', 'wb')
pickle.dump(clf, f)
f.close()
f = open('my_vectorizer.pickle', 'wb')
pickle.dump(vectorizer, f)
f.close()

In [13]:
data=[]
master_data=open(path_test_data,'r')
for sent in master_data:
    data.append(' '.join(sent.split()))
df=pd.DataFrame(data)
df=df[0]

clean_test_data = []
for i in df:
    clean_test_data.append(i)

test_data_features = vectorizer.transform(clean_test_data)
np.asarray(test_data_features)

tfidf_transformer = TfidfTransformer()
X_test_tfidf = tfidf_transformer.fit_transform(test_data_features)

result = clf.predict(X_test_tfidf)

In [14]:
final_result=[]
for idx,prediction in enumerate(result):
    if (prediction=='Not Found'):
        final_result.append('Not Found') 
    else:
        final_result.append(extractions_from_regex[idx])

In [15]:
final_output = pd.DataFrame(data={"sent": df, "label": final_result})
final_output.to_csv(('submission.csv'), index=False, quoting=3, escapechar='\\')

In [16]:
"Done !!!!!!!!!!!"

'Done !!!!!!!!!!!'